## チュートリアル説明

本チュートリアルでは、最終的にニュースコーパスから特徴量を抽出することを目的とする。  
各章では、特徴量を抽出するにあたって、使用しているデータがどのような特性を持つかを解析し、必要に応じて正規化し、可視化する方法を説明する。  
最後の章では、自然語処理モデルであるBERT(Bidirectional Encoder Representations from Transformers)を用いて、特徴量を抽出する方法を説明する。また、抽出された特徴量をLSTMを用いて週ごとに合成する方法を説明する。

## 1.3. 基本準備
本章では、必要なライブラリーと分析用辞書などをインストールし、チュートリアルのコードを正しく利用できる環境を構築する。  
また、本チュートリアルで利用するライブラリーを宣言し、その他、チュートリアルを進めるにあたって必要な基本的な準備を行う。

In [ ]:
# Google Colab環境ではGoogle Driveをマウントしてアクセスできるようにします。
import sys

if 'google.colab' in sys.modules:
    # Google Drive をマウントします
    from google.colab import drive
    mount_dir = "/content/drive"
    drive.mount(mount_dir)

### 1.3.3. 必要なライブラリのインストール

In [ ]:
!apt-get update
!apt-get install -y build-essential sudo mecab libmecab-dev mecab-ipadic-utf8 fonts-ipafont-gothic file
!pip install --no-cache-dir pandas==1.1.5 numpy==1.19.5 scattertext==0.1.0.0 wordcloud==1.8.1 torch==1.7.1 torchvision==0.8.2 transformers==4.2.2 mecab-python3==0.996.6rc1 ipadic==1.0.0 neologdn==0.4 fugashi==1.0.5 japanize-matplotlib==1.1.3 gensim==3.8.3 pyLDAvis==2.1.2

# mecab用の辞書をインストール
!git clone https://github.com/neologd/mecab-ipadic-neologd.git --branch v0.0.7 --single-branch
!yes yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd

### 1.3.4. ライブラリの読み込み

In [ ]:
# 基本ライブラリ
import re
import os
import sys
import math
import random
import json
import joblib
import numpy as np
import pandas as pd
from scipy import stats
import string
from copy import copy
from glob import glob
from itertools import chain
import gc

# テキスト解析関連
import MeCab
import unicodedata
import neologdn

# 可視化関連
from tqdm.auto import tqdm
from IPython.display import display, display_markdown, IFrame
import scattertext as st
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import gensim
import pyLDAvis
import pyLDAvis.gensim

#PCA関連
from sklearn.decomposition import PCA, KernelPCA

# ニューラルネット関連
import torch
from torch import nn
import torch.nn.functional as F
import transformers
from transformers import BertJapaneseTokenizer
from torch.utils.data import DataLoader, Dataset as _Dataset

# ノートブック上でpyLDAvisより可視化を行う場合の設定
pyLDAvis.enable_notebook()

### 1.3.6. 実行環境の確認

In [ ]:
print(sys.version)

### 1.3.7. ファイルパスの設定

In [ ]:
# colab環境で実行する場合
if 'google.colab' in sys.modules:
    CONFIG = {
        'base_path': f'{mount_dir}/MyDrive/JPX_competition/workspace',
        'article_path': f'{mount_dir}/MyDrive/JPX_competition/data_dir_comp2/nikkei_article.csv.gz',
        'stock_price_path': f'{mount_dir}/MyDrive/JPX_competition/data_dir_comp2/stock_price.csv.gz',
        'stock_list_path': f'{mount_dir}/MyDrive/JPX_competition/data_dir_comp2/stock_list.csv.gz',
        'dict_path': '/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd',
        'font_path': '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf',
    }
else:
    CONFIG = {
        'base_path': '/notebook/workspace',
        'article_path': '/notebook/data_dir_comp2/nikkei_article.csv.gz',
        'stock_price_path': '/notebook/data_dir_comp2/stock_price.csv.gz',
        'stock_list_path': '/notebook/data_dir_comp2/stock_list.csv.gz',
        'dict_path': '/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd',
        'font_path': '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf',
    }

### 1.3.8. ディレクトリ作成

In [ ]:
for store_dir in ['headline_features', 'keywords_features', 'visualizations']:
    os.makedirs(os.path.join(CONFIG["base_path"], store_dir), exist_ok=True)

### 1.3.9. テキスト解析用のtaggerをビルド

In [ ]:
# ニュースコーパスの分析のため、taggerをビルドする。
# owakatiはテキストのトークナイズに使用され、ochasenは品詞情報を取得するため使用される。
owakati = MeCab.Tagger(f"-Owakati -d {CONFIG['dict_path']}")
ochasen = MeCab.Tagger(f"-Ochasen -d {CONFIG['dict_path']}")

# taggerのparseを使うことで、各々の機能を確認することができます。
text = 'taggerの役割を確認してみます。'
print('owakati:\n' + owakati.parse(text))
print('ochasen:\n' + ochasen.parse(text))

### 1.3.10. 本番提出用のクラス作成

In [ ]:
# 本番提出用のクラスを作成するため、関数の持たない基本クラスを定義する
# 下記で段階的にクラスを作り上げる
class SentimentGenerator(object):
    # 以下は使用時にビルドされる。
    # 各々に関しては、以下のチュートリアルで説明する。
    article_columns = None
    punctuation_replace_dict = None
    punctuation_remove_list = None
    device = None
    feature_extractor = None
    headline_feature_combiner_handler = None
    keywords_feature_combiner_handler = None

## 1.4. データセットの読み込み
本章では、使用する生データの中で利用可能な情報を把握し、それらの情報だけを取得することを目標とする。

### 1.4.1. データの構成の把握

In [ ]:
# 今回チュートリアルで用いる生データを確認してみる。
articles = pd.read_csv(CONFIG['article_path'])
display(articles.head(3))

In [ ]:
# key: column, value: 重複を排除したデータ件数のdict型宣言
n_unique = {}
for column in articles.columns:
    # column毎の重複を排除したデータ件数をn_uniqueに追加する。
    n_unique[column] = len(articles[column].unique())

display_markdown('Number of unique data', raw=True)

# n_uniqueをpd.Series形式に変換する。articles.dtypesよりcolumnごとのdtypeを取得
# これらをpd.concatによってテーブル形式に変換し、表示
display(pd.concat([pd.Series(n_unique).rename('n_unique'), articles.dtypes.rename('dtype')], axis=1))

In [ ]:
# headlineとkeywordsはニュースの固有情報を含んでいる模様。
# classificationsはその記事の分類、sector情報などが含まれているが、これらはコーパス分析やその特徴量として用いることに適していない。
# company_g.stock_codeは関連する株の情報を含むが、これらはコーパス分析やその特徴量として用いることに適していない。
# classificationsやcompany_g.stock_codeは、モデル学習時のオブジェクト設定や、学習データの生成時のフィルターなどで用いることが望ましい。
# 上記から、本チュートリアルでは、headlineとkeywordsを用いて、分析及び特徴量抽出を行う。

# 表示するcolumnを定義する。
columns = ['headline', 'keywords', 'classifications', 'company_g.stock_code']
for column in columns:
    display_markdown(f'#### {column}', raw=True)

    # 欠損値が含まれることがあるため、どちらかに欠損値が存在するデータを除去する。
    # 同様のindexを持つ5個のサンプルデータを表示。
    display(articles[columns].dropna()[column].head(5))

In [ ]:
# 二つのデータheadline, keywordsとその時刻の情報をもつpublish_datetimeを用いて分析を進め、最終的に、それらのテキストが持つ特徴量を抽出する。
articles = articles[['publish_datetime', 'headline', 'keywords']]
# display(articles.head(3))

In [ ]:
# ニュースが公開された時刻を維持するため、'publish_datetime'をindexとしてセット
articles = articles.set_index('publish_datetime')

# str形式のdatetimeをpd.Timestamp形式に変換
articles.index = pd.to_datetime(articles.index)
# display(articles.head(3))

In [ ]:
# 生データから使用するコラムを設定する
SentimentGenerator.article_columns = ['publish_datetime', 'headline', 'keywords']

# 上記のコードを用いて、本番提出用のクラスにclassmethodを追加
@classmethod
def load_articles(cls, path, start_dt=None, end_dt=None):
    # csvをロードする
    # headline、keywordsをcolumnとして使用。publish_datetimeをindexとして使用。
    articles =  pd.read_csv(path)[cls.article_columns].set_index('publish_datetime')

    # str形式のdatetimeをpd.Timestamp形式に変換
    articles.index = pd.to_datetime(articles.index)

    # 必要な場合、使用するデータの範囲を指定する
    return articles[start_dt:end_dt]


# SentimentGeneratorに定義したclassmethodを追加する
SentimentGenerator.load_articles = load_articles

# SentimentGenerator使用する全体流れを記述
articles = SentimentGenerator.load_articles(path=CONFIG["article_path"])

## 1.5. データセットの前処理
日本語のテキストは半角と全角が混在して使われる場合がある。
同じテキスト内において、同じ単語が半角と全角で混在して使われる場合、人はそれらは単語が同様のであることを認識できるが、モデルにおいては、異なる別の単語として認識される。 更に、半角と全角が混在する単語が学習時に観察していないOut-of-Vocabularyの単語である場合は、その意味が失われ、されにテキスト全体の意味が崩れることにもなり得る。 このような問題を防止するため、テキストの正規化が用いられる。  
本章では、使用するテキストがどのよう文字や記号で構成されていて、どのような期待していない文字や記号などを含むかを調べる。  
更に、それらの期待していない情報を置換や取り除くなどの正規化を行う。

### 1.5.1. 欠損値除去

In [ ]:
# 欠損値の確認は、そのデータを理解する上で一番基本的な方法である。連続したデータの中で欠損値が存在する場合は補正を行う場合があるが、こちらのデータには適していない。
# 欠損値が存在する場合、そのことに留意し、NaN値を取り除くなどの作業が必要となる。
# 本チュートリアルでは、マケットの推定にheadline及びkeywords両方の特徴量を同時に用いる。よって、片方が存在しない場合は、無意味なデータとみなし除去する。

# keywordsはデータに欠損がある
articles.isnull().any()

In [ ]:
# 欠損値を取り除く
articles = articles.dropna()
articles.isnull().any()

### 1.5.2. 全角文字の確認

In [ ]:
# 全角スペース、全角アルファベット、全角数字が含まれているかを確認する
for column in articles.columns:
    for check_target in ["\u3000", r"[Ａ-Ｚａ-ｚ]", r"[０-９]"]:
        display(f'Coulmn: {column}, Contains {check_target}: {articles[column].str.contains(check_target).any()}')

In [ ]:
# Escape codeが含まれているかを確認する。
for column in articles.columns:
    for check_target in ["\t", "\n"]:
        display(f'Coulmn: {column}, Contains {check_target}: {articles[column].str.contains(check_target).any()}')

In [ ]:
# 全角スペースを持つケースを表示
display(articles['headline'][articles['headline'].str.contains('\u3000')][0])
display(articles['keywords'][articles['keywords'].str.contains('\u3000')][0])

In [ ]:
# 全角の英字が含まれケースを表示
display(articles['headline'][articles['headline'].str.contains(r"[Ａ-Ｚａ-ｚ]")][0])
display(articles['keywords'][articles['keywords'].str.contains(r"[Ａ-Ｚａ-ｚ]")][0])

In [ ]:
# 全角の数字が含まれケースを表示
display(articles['headline'][articles['headline'].str.contains(r"[０-９]")][0])
display(articles['keywords'][articles['keywords'].str.contains(r"[０-９]")][0])

### 1.5.3. neologdnによる汎用テキスト正規化
neologdnを用いて、全角英字、数字や、半角カタカナ、一部記号などのを辞書中にマッチできるよう正規化を行う。  
neologdnの正規化規則に関して詳しい情報が必要な場合は次を参照: https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja

In [ ]:
# 以下のテキストをneologdnより正規化する。
text = '全角アルファベット:Ａ, 全角数字:０, 全角スペース:　, 半角カナ:ｱ'

# 正規化前を確認する。
display(text)

In [ ]:
# 正規化後を確認する
display(neologdn.normalize(text))

In [ ]:
for column in articles.columns:
    # スペース(半角スペースを除く、全角スペースやescape codeを含む)はneologdn正規化時に全て除去される。
    # ここでは、スペースの情報が失われないように、スペースを全て改行に書き換え、正規化後スペースに再変換する。
    articles[column] = articles[column].apply(lambda x: '\n'.join(x.split()))

    # neologdnを使って正規化を行う。
    articles[column] = articles[column].apply(lambda x: neologdn.normalize(x))
    
    # 改行をスペースに置換する。
    articles[column] = articles[column].str.replace('\n', ' ')

# ニュース記事の変換前を確認する。
display(articles.head(5))

In [ ]:
# 上記のコードを用いて、本番提出用のクラスにclassmethodを追加
@classmethod
def normalize_articles(cls, articles):
    articles = articles.copy()

    # 欠損値を取り除く
    articles = articles.dropna()

    for column in articles.columns:
        # スペース(全角スペースを含む)はneologdn正規化時に全て除去される。
        # ここでは、スペースの情報が失われないように、スペースを全て改行に書き換え、正規化後スペースに再変換する。
        articles[column] = articles[column].apply(lambda x: '\n'.join(x.split()))

        # neologdnを使って正規化を行う。
        articles[column] = articles[column].apply(lambda x: neologdn.normalize(x))

        # 改行をスペースに置換する。
        articles[column] = articles[column].str.replace('\n', ' ')
        
    return articles

# SentimentGeneratorに定義したclassmethodを追加する
SentimentGenerator.normalize_articles = normalize_articles

# SentimentGenerator使用する全体流れを記述
articles = SentimentGenerator.load_articles(path=CONFIG["article_path"])
articles = SentimentGenerator.normalize_articles(articles)

### 1.5.4. テキスト内の記号情報の扱い
テキスト内には、少なからず記号が使われる場合があるが、希少な記号や正しくない記号の使い方は、モデルがそのテキストを理解する上でのノイズとなり得る。
本章では、コーパス全体に含まれる記号を取得し、それらの記号がどのように使われているかを確認する。また、意味薄い希少な記号は取り除き、正規化の必要な記号は置換する作業を行う。

In [ ]:
# 記号の情報を取得するにあたって、ochasenを用いてテキストから品詞情報を取得してみる。
def parse_by_ochasen(tagger, text):
    # Ochasenでmecab-ipadic-neologdの辞書を使ったときの、返り値のデータ順は以下となる。
    columns = ['表層形', '発音', '原型', '形態素の品詞型', '活用形', '活用型']

    # Ochasenよりコーパスタグ付けを行う。
    parsed = [item.split('\t') for item in tagger.parse(text).split("\n") if item not in ('EOS', '')]
    return pd.DataFrame(parsed, columns=columns)

# 形態素の品詞型より品詞情報を確認すると、記号が記号として正しく認識されていることがわかる。
text = 'テストテキストです。「記号を探してみます!」'
parsed = parse_by_ochasen(tagger=ochasen, text=text)
display(parsed)

In [ ]:
# 記号として扱う品詞型を定義する。MeCabでは、記号の一部が 'サ変接続'に含まれることがある。
# 'サ変接続'の中には、ひらがな、カタカナを含む記号でないものもあるので、それらは、punctuationとして扱わない。
# 以下のflagに該当する品詞型を持つものだけを取得する。
flags = ["記号", "サ変接続"]

# ひらがな、カタカナ、漢字、アルファベットを含まない、記号を全て取得。
punctuation_candidate = parsed['表層形'][parsed['形態素の品詞型'].apply(lambda x: any([flag in x for flag in flags]))]
punctuations = punctuation_candidate[~punctuation_candidate.str.contains(r"[一-龯ぁ-んァ-ンA-Za-z々ゝゞヽヾヴヵヶ]")]
punctuations = set(punctuations)

display(punctuations)

In [ ]:
# 複合記号から単一記号を抽出し、集合に追加する。
for punctuation in punctuations:
    punctuations = punctuations | set(punctuation)
    
display(set(punctuations))

In [ ]:
# コーパス全体から記号を取得するコードを作成
def build_punctuations(tagger, texts, flags = ["記号", "サ変接続"]):
    gc.collect()
    # textsがpd.Seriesでない時に、pd.Seriesに変換
    if isinstance(texts, pd.Series) is False:
        texts = pd.Series(texts)

    punctuations = set()

    for text in tqdm(texts):
        # Ochasenより単語の品詞情報を取得
        parsed = parse_by_ochasen(tagger=tagger, text=text)

        # ひらがな、カタカナ、漢字、アルファベットを含まない、記号を全て取得
        punctuation_candidate = parsed['表層形'][parsed['形態素の品詞型'].apply(lambda x: any([flag in x.split('-') for flag in flags]))]
        punctuation_candidate = punctuation_candidate[~punctuation_candidate.str.contains(r"[一-龯ぁ-んァ-ンA-Za-z々ゝゞヽヾヴヵヶ]")]
        punctuations = punctuations | set(punctuation_candidate.tolist())

    # 複合記号から単一記号を抽出
    for punctuation in punctuations:
        punctuations = punctuations | set(punctuation)

    return punctuations

headline_punctuations = build_punctuations(tagger=ochasen, texts=articles['headline'])
keywords_punctuations = build_punctuations(tagger=ochasen, texts=articles['keywords'])

In [ ]:
# headlineから、これら記号を含むテキストを表示
for punctuation in sorted(headline_punctuations, key=lambda x: len(x)):
    print(f"punctuation: {punctuation}\n", articles['headline'][articles['headline'].apply(lambda x: punctuation in x)][0], '\n')

In [ ]:
# keywordsから、これら記号を含むテキストを表示
for punctuation in sorted(keywords_punctuations, key=lambda x: len(x)):
    print(f"punctuation: {punctuation}\n", articles['keywords'][articles['keywords'].apply(lambda x: punctuation in x)][0], '\n')

In [ ]:
# 上記の観察から、あまり意味の保たない星や音符などの記号は取り除く。見栄えを良くするため乱用に使われている括弧は置換するなど正規化を行う。
# また、多くの機種依存文字が記号が観測されているが、unicode正規化ライブラリーを用いて正規化を行う。

# 機種依存文字の第一水準、第二水準漢字に関しては、名前などに多く使われる一部の漢字を以下で定義し、置換を行う。
JISx0208_replace_dict = {
    '髙': "高",
    '﨑': "崎",
    '濵': "浜",
    '賴': "頼",
    '瀨': "瀬",
    '德': "徳",
    '蓜': "配",
    '昻': "昂",
    '桒': "桑",
    '栁': "柳",
    '犾': "犹",
    '琪': "棋",
    '裵': "裴",
    '魲': "鱸",
    '羽': "羽",
    '焏': "丞",
    '祥': "祥",
    '曻': "昇",
    '敎': "教",
    '澈': "徹",
    '曺': "曹",
    '黑': "黒",
    '塚': "塚",
    '閒': "間",
    '彅': "薙",
    '匤': "匡",
    '冝': "宜",
    '埇': "甬",
    '鮏': "鮭",
    '伹': "但",
    '杦': "杉",
    '罇': "樽",
    '柀': "披",
    '﨤': "返",
    '寬': "寛",
    '神': "神",
    '福': "福",
    '礼': "礼",
    '贒': "賢",
    '逸': "逸",
    '隆': "隆",
    '靑': "青",
    '飯': "飯",
    '飼': "飼",
    '緖': "緒",
    '埈': "峻",
}

# 置換すべき記号のdictionaryを作成する
punctuation_replace_dict = {
    **JISx0208_replace_dict,
    '《': '〈',
     '》': '〉',
     '『': '「',
     '』': '」',
     '“': '"',
     '!!': '!',
     '〔': '[',
     '〕': ']',
     'χ': 'x'
}

# あまり意味を持たない記号のリストを作成し、これらを下記のコードで取り除く
punctuation_remove_list = ['|', '■', '◆', '●', '★', '☆', '♪', '〃', '△', '○', '□']

In [ ]:
# 置換するために定義した機種依存文字の第一水準、第二水準漢字がデータの中に含まれているかチェックします。
# 観測された漢字をstoreするためのsetを定義
catched_replace_targets = set()
for column in articles.columns:
    display_markdown(f'column: {column}', raw=True)
    
    # 定義した置換すべき漢字がデータに含まれているかをチェック
    for key in JISx0208_replace_dict.keys():
        
        # articles[column]にその漢字が含まれている場合、catched_replace_targetsに追加する。
        if articles[column].str.contains(key).any():
            catched_replace_targets.update(key)
            
    display(catched_replace_targets)

In [ ]:
# 機種依存文字の丸囲みの数字、ローマ数字、単位、省略文字などは、unicodedataライブラリーを用いたunicode正規化より置換を行う。
# 正規化形式に関しては本章ではNFKC(Normalization Form Compatibility Composition)を用いる。

# unicodedata.normalize関数より、unicode正規化を行う。
text = '丸囲みの数字:①, ローマ数字:Ⅷ, 単位:㎜㍉, 省略文字:㈱'
unicodedata.normalize('NFKC', text)

In [ ]:
for column in articles.columns:
    # punctuation_remove_listに含まれる記号を除去する
    articles[column] = articles[column].str.replace(fr"[{''.join(punctuation_remove_list)}]", '')
    
    # punctuation_replace_dictに含まれる記号を置換する
    for replace_base, replace_target in punctuation_replace_dict.items():
        articles[column] = articles[column].str.replace(replace_base, replace_target)
                                                    
    # unicode正規化を行う
    articles[column] = articles[column].apply(lambda x: unicodedata.normalize('NFKC', x))

# 精製後確認
articles.head(10)

In [ ]:
# 上で作成した記号置換用のdictと記号削除用のリストをhandle_punctuations_in_articles関数内で使用するため、SentimentGeneratorに追加
SentimentGenerator.punctuation_remove_list = punctuation_remove_list
SentimentGenerator.punctuation_replace_dict = punctuation_replace_dict

# 上記のコードを用いて、本番提出用のクラスにclassmethodを追加
@classmethod
def handle_punctuations_in_articles(cls, articles):
    articles = articles.copy()

    for column in articles.columns:
        # punctuation_remove_listに含まれる記号を除去する
        articles[column] = articles[column].str.replace(fr"[{''.join(cls.punctuation_remove_list)}]", '')

        # punctuation_replace_dictに含まれる記号を置換する
        for replace_base, replace_target in cls.punctuation_replace_dict.items():
            articles[column] = articles[column].str.replace(replace_base, replace_target)

        # unicode正規化を行う
        articles[column] = articles[column].apply(lambda x: unicodedata.normalize('NFKC', x))

    return articles
    
# SentimentGeneratorに定義したclassmethodを追加する
SentimentGenerator.handle_punctuations_in_articles = handle_punctuations_in_articles
                                                        
# SentimentGenerator使用する全体流れを記述
articles = SentimentGenerator.load_articles(path=CONFIG["article_path"])
articles = SentimentGenerator.normalize_articles(articles)
articles = SentimentGenerator.handle_punctuations_in_articles(articles)

## 1.6. データセットの可視化
データセットの解析及び可視化は、データ自体の特性を理解する上で役に立つだけでなく、期待していない情報を含んでいるかをわかる上でも役に立つ。本章では、扱っているコーパスがどのような特性を持つかを、品詞情報や単語の頻度により解析及び可視化を行い、観察結果に応じてデータセットに追加的な処理を行う。

### 1.6.1. 品詞情報取得
下記の続く解析で、多くのコードが品詞情報を要する。しかし、品詞情報の取得には少なからずの時間が所要されるため、一度コーパス全体において品詞情報を取得しておく戦略を取る。

In [ ]:
# コーパス全体をochasenによってparseする。
# メモリー節約のため、解析で使用する['表層形', '形態素の品詞型']の情報のみを残す。
parsed_headline_by_ochasen = articles['headline'].apply(lambda x: parse_by_ochasen(tagger=ochasen, text=x)[['表層形', '形態素の品詞型']])
parsed_keywords_by_ochasen = articles['keywords'].apply(lambda x: parse_by_ochasen(tagger=ochasen, text=x)[['表層形', '形態素の品詞型']])

In [ ]:
display(parsed_headline_by_ochasen.head())
display(parsed_headline_by_ochasen[0])

In [ ]:
display(parsed_keywords_by_ochasen.head())
display(parsed_keywords_by_ochasen[0])

### 1.6.2. 品詞情報解析
本章ではコーパス全体における品詞の分布を解析する。

In [ ]:
# コーパス全体の品詞分布を調べる前に先立ち、単一テキストから品詞分布を取得してみよう。
text = 'テストテキスト。コーパスの品詞情報を取得します。'

# 品詞情報を取得するため、ochasenを用いる
parsed = parse_by_ochasen(tagger=ochasen, text=text)
parsed['形態素の品詞型']

In [ ]:
# 形態素の品詞型は多重のクラスとして成り立っている、ここでは、そのうち一番大きいくくりだけを取得する。
word_classes = parsed['形態素の品詞型'].apply(lambda x: x.split('-')[0])
word_classes

In [ ]:
# 品詞ごとの数をカウントする。
word_classes.groupby(word_classes).count().to_dict()

In [ ]:
# テキストごとの品詞カウントを累積し、コーパス全体の品詞カウントを得るため、カウントをdictionaryにアップデートする
count_of_word_classes = {}
# key: 品詞型, value: カウント数
for key, value in word_classes.groupby(word_classes).count().to_dict().items():
    # dictionaryに品詞が存在しない場合、新しく追加する。
    if key not in count_of_word_classes:
        count_of_word_classes[key] = value

    # dictionaryに品詞が存在する場合、既存のカウントに観察されたカウントを足す。
    else:
        count_of_word_classes[key] += value

# 確認する
count_of_word_classes

In [ ]:
# 上のコードをまとめてコーパス全体から品詞情報を取得する関数を作成。
# コーパスの入力ではなく、時間短縮のため、すでにochasenよりparseされた情報を用いる。
def get_count_of_word_classes(parsed_by_ochasen):
    gc.collect()

    count_of_word_classes = {}

    for parsed in tqdm(parsed_by_ochasen):
        # 一番大きいくくりの品詞型だけを取得する。
        word_classes = parsed['形態素の品詞型'].apply(lambda x: x.split('-')[0])

        # 単一テキストの品詞カウントをdictionaryにアップデート
        # key: 品詞型, value: カウント数
        for key, value in word_classes.groupby(word_classes).count().to_dict().items():
            # dictionaryに品詞が存在しない場合、新しく追加する。
            if key not in count_of_word_classes:
                count_of_word_classes[key] = value

            # dictionaryに品詞が存在する場合、既存のカウントに観察されたカウントを足す。
            else:
                count_of_word_classes[key] += value

    return pd.Series(count_of_word_classes).sort_values()

In [ ]:
# headline, keywords各々のコーパスから品詞情報を取得する。
headline_count_of_word_classes = get_count_of_word_classes(parsed_by_ochasen=parsed_headline_by_ochasen)
keywords_count_of_word_classes = get_count_of_word_classes(parsed_by_ochasen=parsed_keywords_by_ochasen)

In [ ]:
# headlineとkeywordsの品詞の分布がかなり違っていることからそれらの特性を理解することができる。
count_of_word_classes_df = pd.concat([headline_count_of_word_classes.rename('headline'), keywords_count_of_word_classes.rename('keywords')], axis=1, sort=True)
count_of_word_classes_df

In [ ]:
# 比較のため、各々の単語数で各々を割る
normalized_count_of_word_classes_df = count_of_word_classes_df / count_of_word_classes_df.sum()

display(normalized_count_of_word_classes_df.applymap('{:,.6f}'.format))

# keywordsは名詞の割合が98%以上であり、headlineでも高い割合を占める。
# headlineは、名詞意外にも助詞、動詞の割合が高く、高い割合のテキストが文章構造を構築していると想定される。
# keywordsは、名詞が割合が相当高い。コラム名どおり、関連するトピックなどのような名詞情報が含まれていると想定されるが、その他の品詞情報を含まれていることから、少ない数だけれど、文章構造に近いキーワードも入っていることが想定される。
# headlineには記号がかなり含まれていることが観察されるが、記号に対する正規化や削除の前処理を追加的に行うことも考えられる。その際に有意味な情報を持つ記号を削除しないように注意。

In [ ]:
# 上記のテーブルだけでは各々の違いを理解し難いので、可視化を行う。
# データの不均等が激しいため、truncated barplotとして可視化する

# 上下切断表示のため、縦二つのaxesを用意する。
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(12, 5))

# 上下両方のaxesに同様のプロットを行う。
normalized_count_of_word_classes_df.sort_values('headline').plot(kind='bar', ax=ax[0])
normalized_count_of_word_classes_df.sort_values('headline').plot(kind='bar', ax=ax[1])

# 上下両方のy軸範囲を設定する。
ax[0].set_ylim((normalized_count_of_word_classes_df).max().max() - 0.9, 1)
ax[1].set_ylim(0, 0.03)

# 上のaxesでは、bottom部分表示(ticker, labelなど)、下のaxesでは、top部分表示を除去する。
ax[0].spines['bottom'].set_visible(False)
ax[1].spines['top'].set_visible(False)

ax[0].xaxis.tick_top()
ax[0].tick_params(labeltop=False) 
ax[1].xaxis.tick_bottom()

# プロット切断部に点線表示を行う。
d = 0.01  
kwargs = dict(transform=ax[0].transAxes, color='k',linestyle=':', lw=1, clip_on=False)
ax[0].plot((-d, 1+d), (0, 0), **kwargs)    

kwargs.update(transform=ax[1].transAxes)  
ax[1].plot((-d, 1+d), (1, 1), **kwargs)

# 上のaxesだけにlegendを表示するため、下のaxesではlegendを除去する。
ax[1].legend().remove()

plt.tight_layout()
plt.xticks(rotation=0)
plt.show()

### 1.6.3. 頻度解析
一つのテキストや、コーパス全体を可視化する方法には、主にそのコーパスに含まれる単語や形態素の頻度による可視化がある。
ここではwordcloudとscatter textを用いたコーパス全体における単語の頻度の情報を可視化する。

In [ ]:
# 単語の可視化を行う前に当たって、どのコーパスにおいても多く登場するが、あまり意味を持たず、分析において役に立たない不用語(stopwords)を取得し、可視化からそれらを排除する。
# 本解析では、名詞、動詞、形容詞、副詞以外を不用語として扱う。
# 名詞である場合でも、数の情報を含む場合は不用語として扱う。

In [ ]:
# build_punctuationsを少し変更し、stopwordsを取得するコードを作成する
# コード内のひとつの違いは、non_skip_flagsのハンドルが追加されていることである。
# 仮に品詞情報に数が含まれる場合、これらをstopwordsとして扱う。
# コーパスの入力ではなく、時間短縮のため、すでにochasenよりparseされた情報を用いる。

def build_stopwords(parsed_by_ochasen, non_skip_flags=["数"], skip_flags = ["名詞", "動詞", "形容詞", "副詞"]):
    gc.collect()
    stopwords = set()
    
    for parsed in tqdm(parsed_by_ochasen):
        # non_skip_flagsが入っているものは全てstopwordsとして扱う
        # それ以外において、skip_flagsをひとつでも含んでいないものをstopwordsとして扱う
        mask_include_non_skip_flags = parsed['形態素の品詞型'].apply(lambda x: any([non_skip_flag in x.split('-') for non_skip_flag in non_skip_flags]))
        mask_exclude_skip_flags = parsed['形態素の品詞型'].apply(lambda x: not any([skip_flag in x.split('-') for skip_flag in skip_flags]))

        #日、月、年度のようなユニット情報を含むものは全てstopwordsとして扱う
        mask_include_unit_info = parsed['表層形'].apply(lambda x: False if re.fullmatch(r'\d+(秒|分|時|日|月|カ月|年|人|ドル|円)', x) is None else True)

        stopword_candidate = parsed['表層形'][mask_include_non_skip_flags | mask_exclude_skip_flags | mask_include_unit_info]
        
        # stopwordsセットにアップデートする。
        stopwords = stopwords | set(stopword_candidate.tolist())
        
    # 追加的に単一アルファベットをstopwordsとして追加する
    stopwords = stopwords | set(string.ascii_lowercase) | set(string.ascii_uppercase)

    # 追加的に単一数字をstopwordsとして追加する
    stopwords = stopwords | set([str(idx) for idx in range(10)])
    
    return stopwords

In [ ]:
headline_stopwords = build_stopwords(parsed_by_ochasen=parsed_headline_by_ochasen)
keywords_stopwords = build_stopwords(parsed_by_ochasen=parsed_keywords_by_ochasen)

In [ ]:
stopwords = headline_stopwords | keywords_stopwords

# 表示のため、リストに変換する。
stopwords_list = sorted(stopwords)

# 異なる特性を持つ色んなstopwordsを表示するためランダムにシャッフルする。
random.Random(0).shuffle(stopwords_list)
print(stopwords_list[:50])

#### wordcloud

In [ ]:
# 単語の頻度順可視化方法にはwordcloudがある。頻度順に字の大きさが異なるため、全体の傾向を直感的にわかりやすい長所があるが、より詳しい解析や知見を得られない短所がある。
# wordcouldの入力期待値は、単語がスペースとして区切りされている一つのテキストである。コーパス全体を入力とするためには、単語がスペースとして区切りされているコーパスをコーパスごとにスペースで繋げる必要がある。
tagger = owakati
words_with_space = articles['headline'].apply(lambda x: tagger.parse(x).strip("\n").rstrip())
words_with_space = ' '.join(words_with_space)
print(words_with_space[:200])

In [ ]:
# 可視化するための関数を定義する
def display_wordcloud(tagger, texts, stopwords, collocations):
    # textsがpd.Seriesでない時に、pd.Seriesに変換
    if isinstance(texts, pd.Series) is False:
        texts = pd.Series(texts)

    # テキストは単語別にスペースで区切りされ、テキストごとはスペースでつながっていることが期待値
    words_with_space = texts.apply(lambda x: tagger.parse(x).strip("\n").rstrip())
    words_with_space = ' '.join(words_with_space)
    
    # wordcloudを表示するため、パラメータを渡しインスタンス化する。
    # collocations=Falseの場合、連語による重複単語が表示されない。
    wordcloud = WordCloud(
        background_color="white",
        font_path=CONFIG['font_path'],
        stopwords=stopwords,
        width=2000,
        height=1000,
        collocations=collocations,
        random_state=0,
    ).generate(words_with_space)

    # 表示サイズを設定し、表示する。
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))
    ax.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

In [ ]:
# headlineのwordcloud
display_wordcloud(tagger=owakati, texts=articles['headline'], stopwords=stopwords, collocations=False)

In [ ]:
# keywordsのwordcloud
display_wordcloud(tagger=owakati, texts=articles['keywords'], stopwords=stopwords, collocations=False)

#### scatter text

In [ ]:
# 頻度基盤の可視化方法の一つには、頻度の情報だけでなく、その単語の一般性を表す使用分布を共に可視化できるscatter textある。
# scatter textをプロットするにあたって、whitespace_nlp_with_sentences関数を使うためには、各々の単語が空白で区切られる必要がある。
# wordcloudとは違い、こちらのコーパス全体を一つとして繋げる必要はない。
# owakatiでトークナイズし、空白で区切る
tokenized = articles['headline'].apply(lambda x: tagger.parse(x).strip("\n").rstrip())
tokenized.head(3)

In [ ]:
parsed = tokenized.apply(st.whitespace_nlp_with_sentences)
parsed.head(3)

In [ ]:
# stopwordsを除去し、プロット膨大となるため、1000回以下観察された単語は除去する。
corpus = st.CorpusWithoutCategoriesFromParsedDocuments(
    parsed.rename('parse').to_frame(), parsed_col='parse'
).build().get_unigram_corpus().remove_terms(stopwords, ignore_absences=True).remove_infrequent_words(minimum_term_count=1000)

In [ ]:
# プロットのため、単語ごとの頻度情報、使用分布情報を取得する
dispersion = st.Dispersion(corpus)
dispersion_df = dispersion.get_df()

# ビルドされた頻度情報及び使用分布情報から、どの基準を用いてプロットするかをX, Xpos, Y, Yposのcolumnsにセットする。
dispersion_df = dispersion_df.assign(
    X=lambda df: df.Frequency,
    Xpos=lambda df: st.Scalers.log_scale(df.X),
    Y=lambda df: df["Rosengren's S"],
    Ypos=lambda df: st.Scalers.scale(df.Y),
)

dispersion_df.head(5)

In [ ]:
def display_scatter_text(tagger, texts, stopwords, filename='out'): 
    gc.collect()
    # textsがpd.Seriesでない時に、pd.Seriesに変換
    if isinstance(texts, pd.Series) is False:
        texts = pd.Series(texts)
    
    # owakatiより、テキストをparseし、単語をスペースより区切り、whitespace_nlp_with_sentencesを適用。
    tokenized = texts.apply(lambda x: tagger.parse(x).strip("\n").rstrip())
    parsed = tokenized.apply(st.whitespace_nlp_with_sentences)

    # bigramsをcorpusから取り除き、stopwordsを取り除き、また、1000回以下で観察された単語は除去する。
    corpus = st.CorpusWithoutCategoriesFromParsedDocuments(
        parsed.rename('parse').to_frame(), parsed_col='parse'
    ).build().get_unigram_corpus().remove_terms(stopwords, ignore_absences=True).remove_infrequent_words(minimum_term_count=1000)
    
    # Dispersion関数をより、単語ごとの頻度情報及び使用分布情報を取得。
    dispersion = st.Dispersion(corpus)
    dispersion_df = dispersion.get_df()
    dispersion_df = dispersion_df.assign(
        X=lambda df: df.Frequency,
        Xpos=lambda df: st.Scalers.log_scale(df.X),
        Y=lambda df: df["Rosengren's S"],
        Ypos=lambda df: st.Scalers.scale(df.Y),
    )
    
    # dataframe_scattertext関数より、可視化したhtmlをビルドできる。
    html = st.dataframe_scattertext(
        corpus,
        plot_df=dispersion_df,
        ignore_categories=True,
        x_label='Log Frequency',
        y_label="Rosengren's S",
        y_axis_labels=['More Dispersion', 'Medium', 'Less Dispersion'],
    )

    # htmlを書き出します。Google Driveの該当箇所に出力されるため、出力されたhtmlファイルをダウンロードし、ブラウザで開いて御覧ください。
    with open(f'{CONFIG["base_path"]}/visualizations/vis_{filename}_scatter.html', 'w') as f:
      f.write(html)

In [ ]:
# headlineのscatter text
# y軸は使用分布(Dispersion)を表す。Dispersionは特定の単語がコーパス内でどれだけ均等に分布するかを表す数値。1に近いほど均等に現れる単語で、0に近いほど特定の分野や部分でのみ現れる。
# x軸はコーパス全体における単語の登場頻度のログスケールを表す。

# 以下のコードを実行するとvisualizations配下にhtmlファイルが作成されます。作成されたファイルをGoogle Driveからダウンロードし、ブラウザで開いて確認ください。
display_scatter_text(tagger=owakati, texts=articles['headline'], stopwords=stopwords, filename='headline')

In [ ]:
# keywordsのscatter text
# 以下のコードを実行するとvisualizations配下にhtmlファイルが作成されます。作成されたファイルをGoogle Driveからダウンロードし、ブラウザで開いて確認ください。
display_scatter_text(tagger=owakati, texts=articles['keywords'], stopwords=stopwords, filename='keywords')

In [ ]:
# 追加的に、品詞ごとにこれらの頻度や使用分布を可視化することができる。
# その場合、コーパス内での、可視化するターゲット品詞以外の全ての単語を削除し、同様のコードを実行することより可視化できる。

### 1.6.4. トピック解析
コーパス全体の特性を理解するために、そのコーパス全体がどのようなトピックで構成されているかを解析するトピックモデリング方法がある。  
潜在ディリクレ配分法(Latent Dirichlet Allocation, LDA)はトピックモデリングの代表的アルゴリズムである。LDAはコーパスがトピックの混合で成り立っていて、そのトピックは確率分布に基盤し単語を生成すると仮定する。LDAはその過程を逆に辿ることより、コーパス全体を構成するトピック情報を取得できる。

In [ ]:
def build_invalid_tokens(parsed_by_ochasen, non_skip_flags=['数'], skip_flags = ["名詞"]):
    gc.collect()
    # ldaの学習時、単語の頻度情報を持つメトリックスが用いられる。できるだけ単語数を減らすため、名詞以外は全て取り除く。
    return build_stopwords(parsed_by_ochasen=parsed_by_ochasen, non_skip_flags=non_skip_flags, skip_flags=skip_flags)

def tokenize_for_lda(tagger, texts, invalid_tokens):
    gc.collect()
    # textsがpd.Seriesでない時に、pd.Seriesに変換
    if isinstance(texts, pd.Series) is False:
        texts = pd.Series(texts)

    # owakatiを用いてトークナイズする
    tokenized = texts.apply(lambda x: tagger.parse(x).split())

    # 上記で定義したinvalid_tokensに含まれないトークンに精製する。
    tokenized = tokenized.apply(lambda x: [token for token in x if token not in invalid_tokens])
    
    return tokenized

In [ ]:
def build_ldamodel(tagger, texts, invalid_tokens, num_topics=10):
    gc.collect()
    tokenized = tokenize_for_lda(tagger=tagger, texts=texts, invalid_tokens=invalid_tokens)

    # 頻度情報をもつ単語辞書を作る
    dictionary = gensim.corpora.Dictionary(tokenized)

    # 生成されたcorpusは(word_id, word_frequency)の情報を持つ
    corpus = [dictionary.doc2bow(text) for text in tokenized]
    
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, chunksize=5000, passes=10, random_state=0)
    
    return ldamodel, corpus

In [ ]:
# 以下のトピック解析コードは高いcomputation resourceが要求され、実行時間がかなり長い。
# そのことに留意し実行するとしよう。
for column in ['headline', 'keywords']:
    parsed_by_ochasen = {
        'headline': parsed_headline_by_ochasen,
        'keywords': parsed_keywords_by_ochasen,
    }[column]

    invalid_tokens = build_invalid_tokens(parsed_by_ochasen=parsed_by_ochasen)
    ldamodel, corpus = build_ldamodel(tagger=owakati, texts=articles[column], invalid_tokens=invalid_tokens)

    display_markdown(f'#### column: {column}', raw=True)
    
    # 推定されたtopicと関連深い上位5つの単語をプリントする
    for topic in ldamodel.print_topics(num_words=5):
        print(topic)
    
    # トピック可視化
    # 可視化したトピックのidが0ではなく1から始まることに注意。
    # 左方の円は、各々の10個のトピックを表す。
    # 各円との距離は、それぞれトピックがどれだけ離れているかを表す。
    vis = pyLDAvis.gensim.prepare(ldamodel, corpus, ldamodel.id2word, sort_topics=False)
    # htmlを書き出します。出力されたhtmlファイルをダウンロードし、ブラウザで開いて御覧ください。
    pyLDAvis.save_html(vis, f'{CONFIG["base_path"]}/visualizations/vis_{column}_lda.html')

### 1.6.5. 解析後データの追加処理
解析によって得られた知見から、場合に応じて、追加的なデータの精製を必要となる。テキスト内にノイズとなりうる単語の規則を観測した場合や、テキスト自体が無意味であるかノイズとなり得る場合などが該当する。上記の解析を通じて、コーパス全体において、人事の単語が非常に高い頻度で観測されているが、人事の内容はマーケットの予測に役に立つと思われない。今回は、人事の内容を含むニュースの割合が全体のどれぐらいになるかを調べ、必要に応じてそれらのニュースを除去する。

In [ ]:
# 上記で行った単語頻度数を基準とした可視化方法wordcloud, scatter textにおいて、'人事'という単語が多く観察されている。
# ここでは、人事の内容はを含むニュース記事を取り除く作業を行う。

# owakatiを用いて、人事の単語を含む記事を除去する方法もあるが、本番環境でのリソースを軽減させるため、単純に文字列から人事を含む全てのニュースの除去を行う。
# headlineとkeywordsそれぞれに「人事」がどれぐらい含まれているかを確認
headline_drop_mask = articles['headline'].str.contains('人事')
keywords_drop_mask = articles['keywords'].str.contains('人事')

print(f'number of 人事 in headline: {len(articles[headline_drop_mask])} / {len(articles)}')
print(f'number of 人事 in keywords: {len(articles[keywords_drop_mask])} / {len(articles)}')

In [ ]:
# headlineもしくは、keywordsどちらかで人事を含むニュース記事のindexマスクを作成。
drop_mask = headline_drop_mask | keywords_drop_mask

# '人事'を含む例を表示する
articles[drop_mask].head()

In [ ]:
# '人事'を含まないニュースだけに精製する。
articles = articles[~drop_mask]

# '人事'を含むニュースが存在するか確認する。
(articles['headline'].str.contains('人事') | articles['keywords'].str.contains('人事')).any()

In [ ]:
# 上記のコードを用いて、本番提出用のクラスにclassmethodを追加
@classmethod
def drop_remove_list_words(cls, articles, remove_list_words=["人事"]):
    articles = articles.copy()

    for remove_list_word in remove_list_words:
        # headlineもしくは、keywordsどちらかでremove_list_wordを含むニュース記事のindexマスクを作成。
        drop_mask = articles["headline"].str.contains(remove_list_word) | articles[
            "keywords"
        ].str.contains(remove_list_word)

        # remove_list_wordを含まないニュースだけに精製する。
        articles = articles[~drop_mask]

    return articles
    
# SentimentGeneratorに定義したclassmethodを追加する
SentimentGenerator.drop_remove_list_words = drop_remove_list_words
                                                        
# SentimentGenerator使用する全体流れを記述
articles = SentimentGenerator.load_articles(path=CONFIG["article_path"])
articles = SentimentGenerator.normalize_articles(articles)
articles = SentimentGenerator.handle_punctuations_in_articles(articles)
articles = SentimentGenerator.drop_remove_list_words(articles)

## 1.7. 特徴量の生成
テキストから特徴量を抽出するためには、単語の頻度基盤、単語の分散表現基盤、言語モデル基盤などの方法が存在する。  
下記では、いくつかの方法を紹介する。単語の頻度基盤には、
1. テキストから登場する各単語の頻度を行列化したDTM, TF-IDF
2. 潜在意味を抽出するトピックモデリング方法LSA, LDA

これらの方法は、単語の頻度行列の使用を基盤としているため、コーパス内に単語が増えれば増えるほど高いコンピュテーションリーソースが要求される。

単語の分散表現基盤には、
1. 中心単語から周辺単語を予測し、前後単語との関係性を基盤に分散表現を構築するWord2Vec(CBOW)
2. 頻度基盤のLSAと予測基盤のWord2Vecを補うとして、頻度と予測両方を基盤に分散表現を構築するGlove
3. 単語をSubword化し、予測基盤で分散表現を構築することよりOut-Of-Vocabularyに頑健なFastText
4. RNN基盤の言語モデルbiLMを用いて、文脈を反映し分散表現を構築するElmo

これらの方法は、単語を分散表現化する方法であるが、モデルに入力とするSparse vectorにテキスト内の単語を全て与えるか、単語分散表現を合成するなどの処理を行い、テキストの分散表現を取得することができる。

言語モデル基盤には(本内容では、ニューラルネット基盤の言語モデルのみを説明する)、
1. 循環神経網(Recurrent Neural Network, RNN)基盤(LSTM, GRU, LMを含む)の内部状態を特徴量として抽出
2. Transformer(BERTなどattention基盤モデルを含む)基盤の内部状態を特徴量として抽出

これらの方法は、膨大なデータの学習受容量を持ち、またそれらの膨大なデータから学習されたモデルは、テキスト内でのより高次元の潜在表現を学習できる。また、それらから抽出した特徴量は多様なタスクにおいて適用でき、且つ、高いパフォーマンスを表すことが知られている。

本章では、BERT(Bidirectional Encoder Representations from Transformers)を用いて、テキストを入力し、そのテキストが持つ潜在意味を特徴量として抽出する方法を説明する。

### 1.7.1. 特徴量抽出機、前処理機の定義

In [ ]:
# 下記では、BERTモデルを扱う上で必要な、特徴量抽出時使うデバイスの定義、特徴量抽出機の定義、BERTの前処理機の定義を行う。

@classmethod
def _set_device(cls):
    # 使用可能なgpuがある場合、そちらを利用し特徴量抽出を行う
    if torch.cuda.device_count() >= 1:
        cls.device = 'cuda'
        print("[+] Set Device: GPU")
    else:
        cls.device = 'cpu'
        print("[+] Set Device: CPU")

@classmethod
def _build_feature_extractor(cls):
    # 特徴量抽出のため事前学習済みBERTモデルを用いる。
    # ここでは、"cl-tohoku/bert-base-japanese-whole-word-masking"モデルを使用しているが、異なる日本語BERTモデルを用いても良い。
    cls.feature_extractor = (
        transformers.BertModel.from_pretrained(
            "cl-tohoku/bert-base-japanese-whole-word-masking",
            return_dict=True,
            output_hidden_states=True,
        )
    )

    # 使用するdeviceを指定
    cls.feature_extractor = cls.feature_extractor.to(cls.device)

    # 今回、学習は行わない。特徴量抽出のためなので、評価モードにセットする。
    cls.feature_extractor.eval()
    
    print("[+] Built feature extractor")

@classmethod
def _build_tokenizer(cls):
    # BERTモデルの入力とするコーパスはそのBERTモデルが学習された時と同様の前処理を行う必要がある。
    # 今回使用する"cl-tohoku/bert-base-japanese-whole-word-masking"モデルは、mecab-ipadic-NEologdによりトークナイズされ、その後Wordpiece subword encoderよりsubword化している。
    # Subwordとは形態素の類似な概念として、単語をより小さい意味のある単位に変換したものである。
    # transformersのBertJapaneseTokenizerは、その事前学習モデルの学習時と同様の前処理を簡単に使用することができる。
    # この章ではBertJapaneseTokenizerを利用し、トークナイズ及びsubword化を行う。
    cls.bert_tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
    print("[+] Built bert tokenizer")
    
# SentimentGeneratorに定義したclassmethodを追加する
SentimentGenerator._set_device = _set_device
SentimentGenerator._build_feature_extractor = _build_feature_extractor
SentimentGenerator._build_tokenizer = _build_tokenizer
                                                        
# SentimentGenerator使用する全体流れを記述
articles = SentimentGenerator.load_articles(path=CONFIG["article_path"])
articles = SentimentGenerator.normalize_articles(articles)
articles = SentimentGenerator.handle_punctuations_in_articles(articles)
articles = SentimentGenerator.drop_remove_list_words(articles)

SentimentGenerator._set_device()
SentimentGenerator._build_feature_extractor()
SentimentGenerator._build_tokenizer()

### 1.7.2. BERTモデルを使用するための前処理

In [ ]:
#　二つの前処理を比較してみる
# SentimentGenerator.bert_tokenizer: mecab-ipadic-NEologd + Wordpiece
# owakati: mecab-ipadic-NEologd

# Subword化を行うbert_tokenizerの方がより小さい単位でトークン化されていることがわかる。
# このようなsubwordを用いると、学習時観察されていない単語(Out of Vocabulary)に関する問題を緩和させることができる。
text = '我らは走り出す。'
display(SentimentGenerator.bert_tokenizer.tokenize(text))
display(owakati.parse(text).split())

In [ ]:
# 基本的にどの言語モデルもトークナイズ後のトークンをそのまま受け取ることはできない。トークンを数字に対応させるid化が必要とある。
# 事前学習モデルを用いる場合、各々のトークンはすでにidが付与されているため、そのidに変更し、モデルの入力として扱う必要がある。
# BertJapaneseTokenizerのencodeはこのようなid化を行ってくれる。
display(SentimentGenerator.bert_tokenizer.encode(text))

In [ ]:
# id化したトークンを再度トークンに変更してみる
for id in SentimentGenerator.bert_tokenizer.encode(text):
    print(f'{id}: {SentimentGenerator.bert_tokenizer.decode(id)}')
    
# idをトークンに再度戻した時、元のデータには存在していない[CLS]と[SEP]を観察することができる。この二つはspecial tokensと呼ばれ、[CLS]は全ての文章の前に使用され出力層におけるtoken sequenceの結合意味からクラス分類を行うため用いられる。[SEP]は複数の文章を区切るために用いられる。
# 入力は必ずこのフォーマットに従う必要がある。

In [ ]:
# BERTの入力値は上記で生成したtokenのids以外に, token_type_ids, attention_maskのベクトルを入力値として受け取る。
# token_type_idsは複数の文章を区切るため用いられる。
# attention maskは実際トークンが存在する部分とzero paddingされた部分を区切るため用いられる。

encoded = SentimentGenerator.bert_tokenizer.encode_plus(
    text,
    None,
    add_special_tokens=True,
    return_token_type_ids=True,
    truncation=True,
)

encoded

In [ ]:
# torchモデルに入力するためにはtensor形式に変え、deviceを指定する必要がある。
input_ids = torch.tensor([encoded['input_ids']], dtype=torch.long).to(SentimentGenerator.device)
token_type_ids = torch.tensor([encoded['token_type_ids']], dtype=torch.long).to(SentimentGenerator.device)
attention_mask = torch.tensor([encoded['attention_mask']], dtype=torch.long).to(SentimentGenerator.device)

display(input_ids)
display(token_type_ids)
display(attention_mask)

### 1.7.3. 特徴量抽出

In [ ]:
# 2.前処理の最後で取得した[input_ids, token_type_ids, attention_mask]三つのベクトルを特徴量抽出機に入力し、BERTモデルのoutputを取る
output = SentimentGenerator.feature_extractor(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

In [ ]:
# outputをdictionary形式に変換し、その中からどのようなkeyが存在するかをみてみる。
output.__dict__.keys()

# last_hidden_stateはモデル最終層出力の全てのsequenceのhidden stateを返す
# pooler_outputはモデル最終層出力の最初のsequenceのhidden stateを返す
# hidden_statesはモデル各層出力の全てのsequenceのhidden stateを返す
# attentionsはattention softmax以降のattentions weightsを返す
# cross_attentionsはdecoderのcross-attention層における、attention softmax以降のattentions weightsを返す

In [ ]:
# Deep neural networkではより後層に行くほど、高次表現を持つことが知られている。ここでは、最終層の一つ前のhidden stateを特徴量として抽出し使用することにする。
# 最終層でなはなく、最終層手前の情報を利用する理由は、最終層は学習タスクにおけるオブジェクトに強く引き寄せられた情報を持つため、一般的には特徴量抽出のために、より豊富な情報を持つ最終層手前使用する。
# この内容に関してより詳しく知りたい場合以下を参照: https://bert-as-service.readthedocs.io/en/latest/section/faq.html#why-not-the-last-hidden-layer-why-second-to-last
features = output['hidden_states'][-2]
display(features)

In [ ]:
# featuresの次元を見ると、[1, 8, 768]次元を持つが、各々順番に、[データ数、シーケンスサイズ、hidden stateのサイズ]を表している。
display(features.size())

In [ ]:
# 一つ注意すべきところが、長さの異なるシークエンスを入力すると以下のようにベクトルのサイズが変わってくる。
text = 'こちらでは、より長い文章を用いて特徴量を抽出してみましょう。'
encoded = SentimentGenerator.bert_tokenizer.encode_plus(
    text,
    None,
    add_special_tokens=True,
    return_token_type_ids=True,
    truncation=True,
)

input_ids = torch.tensor([encoded['input_ids']], dtype=torch.long).to(SentimentGenerator.device)
token_type_ids = torch.tensor([encoded['token_type_ids']], dtype=torch.long).to(SentimentGenerator.device)
attention_mask = torch.tensor([encoded['attention_mask']], dtype=torch.long).to(SentimentGenerator.device)

output = SentimentGenerator.feature_extractor(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
features = output['hidden_states'][-2]
features.size()

# dimension1のサイズが変わっていることがわかる。最終的に、この章ではdimension1の次元を平均化し特徴量として扱うため、シークエンスの違いがそれほど問題とはならない。
# しかし、モデルへの同時入力、並列化する上では問題となる。シーケンスの異なるベクトルを重ねることができないからである。
# このような問題を扱うため、subwordsのシーケンスのmax_lengthを決め、それより短い場合は、max_lengthの長さとなるようにベクトルの末端を0で埋めるzero paddingがよく使われる。

In [ ]:
# 前処理機にmax_lengthのパラメータを渡し、padding='max_legnth'を指定すると、max_lengthの長さがなるようにzero paddingすることができる
text = 'こちらでは、より長い文章を用いて特徴量を抽出してみましょう。'
encoded = SentimentGenerator.bert_tokenizer.encode_plus(
    text,
    None,
    max_length=512,
    padding='max_length',
    add_special_tokens=True,
    return_token_type_ids=True,
    truncation=True,
)

encoded

In [ ]:
# 異なる長さを持つ複数のコーパスを用いて、paddingされたinputを用いてモデルに同時入力してみる。
texts = ['短いテキスト', '少し長いテキストです', '長い長い長い長いテキストです']

input_ids = []
token_type_ids = []
attention_mask = []
for text in texts:
    encoded = SentimentGenerator.bert_tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=512,
        padding="max_length",
        return_token_type_ids=True,
        truncation=True,
    )

    input_ids.append(encoded['input_ids'])
    token_type_ids.append(encoded['token_type_ids'])
    attention_mask.append(encoded['attention_mask'])

input_ids = torch.tensor(input_ids, dtype=torch.long).to(SentimentGenerator.device)
token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).to(SentimentGenerator.device)
attention_mask = torch.tensor(attention_mask, dtype=torch.long).to(SentimentGenerator.device)

output = SentimentGenerator.feature_extractor(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
features = output['hidden_states'][-2]
features.size()

# [3つのコーパス、シーケンスの長さ、hidden state]の次元の特徴量を取得できた。
# dimention1は指定したmax_lengthと一致することがわかる。

In [ ]:
# 最終的には、dimension1を平均化し、各テキストごとに768次元のベクトルを特徴量として抽出する。
features = features.mean(dim=1)
features.size()

In [ ]:
# 上記のコードを纏め、テキストから前処理を行い、モデル入力に必要な各々のベクトルを返す関数を作成。
@classmethod
def build_inputs(cls, texts, max_length=512):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    for text in texts:
        encoded = cls.bert_tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_length,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True,
        )
        
        input_ids.append(encoded['input_ids'])
        token_type_ids.append(encoded['token_type_ids'])
        attention_mask.append(encoded['attention_mask'])
        
    # torchモデルに入力するためにはtensor形式に変え、deviceを指定する必要がある。
    input_ids = torch.tensor(input_ids, dtype=torch.long).to(cls.device)
    token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).to(cls.device)
    attention_mask = torch.tensor(attention_mask, dtype=torch.long).to(cls.device)
    
    return input_ids, token_type_ids, attention_mask

# 上記のコードでビルドした、ベクトルをモデルに入力し、特徴量を抽出するコード作成
@classmethod
def generate_features(cls, input_ids, token_type_ids, attention_mask):
    output = cls.feature_extractor(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
    features = output['hidden_states'][-2].mean(dim=1).cpu().detach().numpy()
    
    return features


# コーパス全体から特徴量を抽出するため、コーパス全体を同時にモデルへ入力することはメモリーの上限を遥かに超えてしまうので不可能に近い。
# 入力するコーパスを数回に分割し、上記で作成したコードbuild_inputsとgenerate_featuresを用いて並列化処理を行うため、以下のコードを作成する。
@classmethod
def generate_features_by_texts(cls, texts, batch_size=2, max_length=512):
    n_batch = math.ceil(len(texts) / batch_size)

    features = []
    for idx in tqdm(range(n_batch)):
        input_ids, token_type_ids, attention_mask = cls.build_inputs(texts=texts[batch_size*idx:batch_size*(idx+1)], max_length=max_length)
        
        features.append(cls.generate_features(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask))
        
    features = np.concatenate(features, axis=0)
    
    # 抽出した特徴量はnp.ndarray形式となっており、これらは、日付の情報を失っているため、pd.DataFrame形式に変換する。
    return pd.DataFrame(features, index=texts.index)

# SentimentGeneratorに定義したclassmethodを追加する
SentimentGenerator.build_inputs = build_inputs
SentimentGenerator.generate_features = generate_features
SentimentGenerator.generate_features_by_texts = generate_features_by_texts

# SentimentGenerator使用する全体流れを記述
articles = SentimentGenerator.load_articles(path=CONFIG["article_path"])
articles = SentimentGenerator.normalize_articles(articles)
articles = SentimentGenerator.handle_punctuations_in_articles(articles)
articles = SentimentGenerator.drop_remove_list_words(articles)

SentimentGenerator._set_device()
SentimentGenerator._build_feature_extractor()
SentimentGenerator._build_tokenizer()


In [ ]:
# 以下のコードでコーパス全体の特徴量を抽出できる。しかし、抽出には長い時間が要求されるため、注意。
# headlineの特徴量を抽出
headline_features = SentimentGenerator.generate_features_by_texts(texts=articles['headline'])

In [ ]:
# pklファイルとしてstoreしておく。
headline_features.to_pickle(f'{CONFIG["base_path"]}/headline_features/headline_features.pkl')

In [ ]:
# keywordsの特徴量を抽出
keywords_features = SentimentGenerator.generate_features_by_texts(texts=articles['keywords'])

In [ ]:
# pklファイルとしてstoreしておく。
keywords_features.to_pickle(f'{CONFIG["base_path"]}/keywords_features/keywords_features.pkl')

# PCAによるスコア取得

In [ ]:
def _build_compressor(compress_method):
    assert compress_method in ('pca', 'kpca')
    if compress_method == 'pca':
        return PCA(n_components=1)
    
    if compress_method == 'kpca':
        return KernelPCA(kernel='rbf', n_components=1)

def compress_feature_n_samples(features, compress_method, max_samples=500):    
    feature_compressor = _build_compressor(compress_method=compress_method)
    compressed_features = pd.Series(feature_compressor.fit_transform(features).reshape(-1), index=features.index)

    sample_compressor = _build_compressor(compress_method=compress_method)

    weekly_group = pd.Series(zip(compressed_features.index.year, compressed_features.index.week), index=compressed_features.index)
    grouped_compressed_features = compressed_features.groupby(weekly_group).apply(lambda x: x[-max_samples:].reset_index(drop=True)).unstack()

    return pd.Series(sample_compressor.fit_transform(grouped_compressed_features).reshape(-1), index=grouped_compressed_features.index)

In [ ]:
for features, feature_type in [(headline_features, 'headline_features'), (keywords_features, 'keywords_features')]:
    for compress_method in ['pca']:
        compress_feature_n_samples(features=features, compress_method=compress_method).to_pickle(f'{feature_type}_method3_{compress_method}.pkl')